# Level up!

Schauen wir nun einige Fortgeschrittene Fälle an:
1. Ein Feld kann auch mehrere Datentypen unterstützen
2. Wir wollen, dass ein Feld einen anderen Namen hat im Pydantic-Model, als es im JSON definiert ist
3. Benutzerdefinierte Validierung

Machen wir das Schritt für Schritt.

__Mehrere Datentypen__

Folgendes Beispiel zeigt, wie ein Pydantic mehrere Typen in einem Field untersützen kann.

In [9]:
from pydantic import BaseModel
from typing import Union

class ShittyAPI(BaseModel):
    value: Union[int, str]
    # more modern syntax in Python3.10
    #>>> value: str | int


input: dict = {'value': 1}
parsed = ShittyAPI.parse_obj(input)
print(f'example 1: {parsed.value} (type={type(parsed.value)})')


another_input: dict = {'value': 'one'}
parsed = ShittyAPI.parse_obj(another_input)
print(f'example 2: {parsed.value} (type={type(parsed.value)})')


example 1: 1 (type=<class 'int'>)
example 2: one (type=<class 'str'>)


__Benutzerdefinierte Felder-Namen__

Wir wollen, dass ein Feld einen anderen Namen hat im Pydantic-Model, als es im JSON definiert ist.

In [11]:
from pydantic import BaseModel, Field


class ShittyAPI(BaseModel):
    properly_named_field: int = Field(..., alias="super_stupid_propertiy_name")

input: dict = {'super_stupid_propertiy_name': 111}
ShittyAPI.parse_obj(input)


ShittyAPI(properly_named_field=111)

__Benutzerdefinierte Validierung__

Wir können auch selbst Validatoren definieren. Dazu schauen wir das offizielle Pydantic Beispiel an:

https://docs.pydantic.dev/latest/usage/validators/


In [15]:
from pydantic import BaseModel, ValidationError, validator


class UserModel(BaseModel):
    name: str
    username: str
    password1: str
    password2: str

    @validator('name')
    def name_must_contain_space(cls, v):
        if ' ' not in v:
            raise ValueError('must contain a space')
        return v.title()

    @validator('password2')
    def passwords_match(cls, v, values, **kwargs):
        if 'password1' in values and v != values['password1']:
            raise ValueError('passwords do not match')
        return v

    @validator('username')
    def username_alphanumeric(cls, v):
        assert v.isalnum(), 'must be alphanumeric'
        return v



Sofern wir alles korrekt eingeben, wird das UserModel erfolgreich initalisiert

In [13]:
user = UserModel(
    name='samuel colvin',
    username='scolvin',
    password1='zxcvbn',
    password2='zxcvbn',
)
print(user)
#> name='Samuel Colvin' username='scolvin' password1='zxcvbn' password2='zxcvbn'



name='Samuel Colvin' username='scolvin' password1='zxcvbn' password2='zxcvbn'


Bei fehlerhaftem Input wirft es hingegen einen Fehler.

In [14]:
try:
    UserModel(
        name='samuel',
        username='scolvin',
        password1='zxcvbn',
        password2='zxcvbn2',
    )
except ValidationError as e:
    print(e)
    """
    2 validation errors for UserModel
    name
      must contain a space (type=value_error)
    password2
      passwords do not match (type=value_error)
    """

2 validation errors for UserModel
name
  must contain a space (type=value_error)
password2
  passwords do not match (type=value_error)
